# Multiple Dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
square(x) = x^2

square (generic function with 1 method)

In [2]:
square(10)

100

In [3]:
square("Hello ")

"Hello Hello "

In [4]:
square([1,2,3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:796
  ^(!Matched::Missing, ::Integer) at missing.jl:130
  ^(!Matched::Missing, ::Number) at missing.jl:94
  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `f` that only takes `Number`s as inputs.

In [5]:
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 1 method)

In [6]:
f(3, 4)

"a and b are both integers"

In [7]:
f(1.2, 3.4)

MethodError: MethodError: no method matching f(::Float64, ::Float64)

But we can define that method!

In [8]:
f(a::Float64, b::Float64) = "a and b are both Float64s"

f (generic function with 2 methods)

In [9]:
f(1.2, 3.4)

"a and b are both Float64s"

### Basic dispatch

In [10]:
f(a, b) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 6 methods)

In [11]:
methods(f)

# 6 methods for generic function "f":
[1] f(a::Float64, b::Float64) in Main at In[8]:1
[2] f(a::Integer, b::Integer) in Main at In[10]:5
[3] f(a::Number, b::Number) in Main at In[10]:2
[4] f(a::Number, b) in Main at In[10]:3
[5] f(a, b::Number) in Main at In[10]:4
[6] f(a, b) in Main at In[10]:1

In [12]:
f(1.5, 2)

"a and b are both numbers"

In [13]:
f(1, "bar")

"a is a number"

In [14]:
f(1, 2)

"a and b are both integers"

In [15]:
f("foo", [1,2])

"fallback"

In [16]:
f(1, 2, 3)

MethodError: MethodError: no method matching f(::Int64, ::Int64, ::Int64)
Closest candidates are:
  f(::Integer, ::Integer) at In[10]:5
  f(::Number, ::Number) at In[10]:2
  f(::Number, ::Any) at In[10]:3
  ...

### Ambiguities

In [17]:
g(a::Int, b::Number) = 1
g(a::Number, b::Int) = 2

g (generic function with 2 methods)

In [18]:
g(1, 2.5)

1

In [19]:
g(1.5, 2)

2

In [20]:
g(1, 2)

MethodError: MethodError: g(::Int64, ::Int64) is ambiguous. Candidates:
  g(a::Number, b::Int64) in Main at In[17]:2
  g(a::Int64, b::Number) in Main at In[17]:1
Possible fix, define
  g(::Int64, ::Int64)

In [21]:
g(x::Int, y::Int) = 3

g (generic function with 3 methods)

In [22]:
g(1, 2)

3

### "Diagonal" dispatch

In [23]:
f(a::T, b::T) where {T<:Number} = "a and b are both $(T)s"

f (generic function with 7 methods)

In [24]:
methods(f)

# 7 methods for generic function "f":
[1] f(a::Float64, b::Float64) in Main at In[8]:1
[2] f(a::Integer, b::Integer) in Main at In[10]:5
[3] f(a::T, b::T) where T<:Number in Main at In[23]:1
[4] f(a::Number, b::Number) in Main at In[10]:2
[5] f(a::Number, b) in Main at In[10]:3
[6] f(a, b::Number) in Main at In[10]:4
[7] f(a, b) in Main at In[10]:1

In [25]:
f(big(1.5), big(2.5))

"a and b are both BigFloats"

In [26]:
f(big(1), big(2)) # <== integer rule is more specific

"a and b are both integers"

In [27]:
f(a::T, b::T) where {T<:Integer} = "both are $T integers"

f (generic function with 8 methods)

In [28]:
methods(f)

# 8 methods for generic function "f":
[1] f(a::Float64, b::Float64) in Main at In[8]:1
[2] f(a::T, b::T) where T<:Integer in Main at In[27]:1
[3] f(a::Integer, b::Integer) in Main at In[10]:5
[4] f(a::T, b::T) where T<:Number in Main at In[23]:1
[5] f(a::Number, b::Number) in Main at In[10]:2
[6] f(a::Number, b) in Main at In[10]:3
[7] f(a, b::Number) in Main at In[10]:4
[8] f(a, b) in Main at In[10]:1

In [29]:
f(big(1), big(2))

"both are BigInt integers"

In [30]:
f("foo", "bar") # <== still doesn't apply to non-numbers

"fallback"

### Varargs methods

In [31]:
f(args::Number...) = "$(length(args))-ary heterogeneous call"
f(args::T...) where {T<:Number} = "$(length(args))-ary homogeneous call"

f (generic function with 10 methods)

In [32]:
f(1)

"1-ary homogeneous call"

In [33]:
f(1, 2, 3)

"3-ary homogeneous call"

In [34]:
f(1, 1.5, 2)

"3-ary heterogeneous call"

In [35]:
f()

"0-ary homogeneous call"

In [36]:
f(1, 2) # <== previous 2-arg method is more specific

"both are Int64 integers"

In [37]:
f("foo") # <== still doesn't apply to non-numbers

MethodError: MethodError: no method matching f(::String)
Closest candidates are:
  f(::Any, !Matched::Number) at In[10]:4
  f(::Any, !Matched::Any) at In[10]:1
  f(!Matched::Float64, !Matched::Float64) at In[8]:1
  ...

In [38]:
# "splat" (more below)
f([1, 2, 3]...)

"3-ary homogeneous call"

### Optional Arguments

In [39]:
h(x, y = 0) = 2x + 3y

h (generic function with 2 methods)

In [40]:
methods(h)

# 2 methods for generic function "h":
[1] h(x) in Main at In[39]:1
[2] h(x, y) in Main at In[39]:1

Shorthand for this:
```
h(x, y) = 2x + 3y
h(x) = h(x, 0)
```

### Keyword Arguments

In [41]:
k(x, y = 0; opt::Bool = false) = opt ? 2x+y : x+2y

k (generic function with 2 methods)

In [42]:
k(2)

2

In [43]:
k(2, 3)

8

In [44]:
k(2, opt=true)

4

In [45]:
k(2, 3, opt=true)

7

In [46]:
foo(x, y; req::Bool) = req ? 2x+y : x+2y

foo (generic function with 1 method)

In [47]:
foo(2, 3)

UndefKeywordError: UndefKeywordError: keyword argument req not assigned

In [48]:
methods(k)

# 2 methods for generic function "k":
[1] k(x) in Main at In[41]:1
[2] k(x, y; opt) in Main at In[41]:1

In [49]:
k(2, opt=true)

4

### Keyword arguments: slurp and splat

In [50]:
function allkw(; kw...)
    @show keys(kw)
end

allkw (generic function with 1 method)

In [51]:
allkw(a=1,b=2)

keys(kw) = (:a, :b)


(:a, :b)

Just like iterators can be splatted as positional arguments, dict-like collections and named tuples can be splatted as keyword arguments.

In [52]:
function rect(;width=1,height=1,fill="#")
    for i in 1:height
        println(fill^width)
    end
end

rect (generic function with 1 method)

In [53]:
params = (width=8,height=3,fill='A')

(width = 8, height = 3, fill = 'A')

In [54]:
rect(; params...)

AAAAAAAA
AAAAAAAA
AAAAAAAA


### Exercises

#### Exercise 1

Write a function that repeats a string an integer number of times which takes the arguments in either order.

#### Exercise 2a

Write a function `F` that returns the tuple `(x, y, k)` where:
- `x` is the first positional argument and is mandatory
- `y` is the second positional argument and is optional
- `k` is an optional keyword argument

The optional arguments should have the following defaults:
- `y` defaults to `2x`
- `k` defaults to `2y`

#### Exercise 2b

Write a function `G` just like `F` but with differet defaults:
- `k` defaults to `2x`
- `y` defaults to `2k`